<a href="https://colab.research.google.com/github/naohiro701/naohiro701/blob/main/movie_to_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# youtube url
youtube_url = ""

### set google drive 

In [ ]:
from google.colab import drive
from google.colab import files
# drive.mount('/content/drive')
# mp3_url = files.upload() #MP3

### convert movie on youtube to mp3

In [ ]:
!python -m pip install -U yt-dlp

In [ ]:
from yt_dlp import YoutubeDL

# 条件設定
ydl_opts = {
    'format': 'bestaudio/best',
    'outtmpl':  "sample_music" + '.%(ext)s',
    'postprocessors': [
        {'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
         'preferredquality': '128'},
        {'key': 'FFmpegMetadata'},
    ],
}

In [ ]:
with YoutubeDL(ydl_opts) as ydl:
    error_code = ydl.download(youtube_url )

### import wisper

In [ ]:
!pip install git+https://github.com/openai/whisper.git 

|  Size  | Parameters | English-only model | Multilingual model | Required VRAM | Relative speed |
|:------:|:----------:|:------------------:|:------------------:|:-------------:|:--------------:|
|  tiny  |    39 M    |     `tiny.en`      |       `tiny`       |     ~1 GB     |      ~32x      |
|  base  |    74 M    |     `base.en`      |       `base`       |     ~1 GB     |      ~16x      |
| small  |   244 M    |     `small.en`     |      `small`       |     ~2 GB     |      ~6x       |
| medium |   769 M    |    `medium.en`     |      `medium`      |     ~5 GB     |      ~2x       |
| large  |   1550 M   |        N/A         |      `large`       |    ~10 GB     |       1x       |


In [ ]:
import whisper

In [ ]:
model = whisper.load_model("large")
mp3_url = "/content/sample_music.mp3"
result = model.transcribe(mp3_url)
# print(result["text"])

### make word cloud

In [ ]:
!apt install aptitude
!aptitude install git make curl xz-utils file -y
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 -y
!pip install mecab-python3==0.7
!apt-get -y install fonts-ipafont-gothic

In [ ]:
import MeCab
from collections import defaultdict
import re
import io
import urllib.request
from zipfile import ZipFile
import IPython
from wordcloud import WordCloud

In [ ]:
def load_from_url(url):
    data = urllib.request.urlopen(url).read()
    zipdata = ZipFile(io.BytesIO(data))
    filename = zipdata.namelist()[0]
    text = zipdata.read(filename).decode("shift-jis")
    text = re.sub(r'［.*?］', '', text)
    text = re.sub(r'《.*?》', '', text)
    return text

In [ ]:
def show_top10(text):
    m = MeCab.Tagger()
    node = m.parseToNode(text)
    dic = defaultdict(int)
    while node:
        a = node.feature.split(",")
        key = node.surface
        if a[0] == u"名詞" and a[1] == u"一般" and key != "":
            dic[key] += 1
        node = node.next
    for k, v in sorted(dic.items(), key=lambda x: -x[1])[0:10]:
        print(k + ":" + str(v))

# show_top10(text)

In [ ]:
def get_words(text):
    w = ""
    m = MeCab.Tagger()
    node = m.parseToNode(text)
    while node:
        a = node.feature.split(",")
        if a[0] == u"名詞" and a[1] == u"一般":
            w += node.surface + " "
        node = node.next
    return w

In [ ]:
text = result["text"]

In [ ]:
words = get_words(text)
fpath='/usr/share/fonts/opentype/ipafont-gothic/ipagp.ttf'
wc = WordCloud(background_color="white", width=4800, height=3200, font_path=fpath)
wc.generate(words)
wc.to_file("wc.png")
IPython.display.Image("wc.png")